In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import re
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from kaggle_utils import move_feature

In [6]:
paths_feim = glob('../output/feature_importances/20190922_2258__CV0-9571362096465762__feature1042.gz') \
# + glob('../output/feature_importances/20190909*.gz') \
# + glob('../output/feature_importances/20190908*.gz') 
# + glob('../output/feature_importances/20190907*.gz') \
# + glob('../output/feature_importances/20190906*.gz')
# paths_feim = glob('../output/selection_feature/*.gz')
list_good_feature = []
list_feim = []

for path in paths_feim:
    try:
        lb_score = re.search(rf'CV([^/.]*)__', path).group(1)
    except AttributeError:
        pass
    if float(lb_score.replace('-', '.'))<0.9167:
        continue

    feim = read_pkl_gzip(path)
#     feim = feim[feim['is_valid']==1].head(1).tail(1)
#     feim['score'] = lb_score
#     list_feim.append(feim)
# df_feim = pd.concat(list_feim, axis=0)
# df_feim.sort_values(by='importance', ascending=False, inplace=True)
# display(df_feim)
#     tmp_list = feim.head(1).feature.tolist()
#     for feature in tmp_list:
#         if feature not in set(list_good_feature):
#             list_good_feature.append(feature)

In [9]:
# from_dir = '../submit/add_feature/'
from_dir = 'valid_use'
to_dir = '../submit/re_sub/'
# for feature in df_feim['feature'].values:
# for feature in feim[feim['is_valid']==1]['feature'].values:
for feature in feim.index:
    move_feature([feature], from_dir, to_dir)

In [27]:
pd.Series(list_good_feature).value_counts()

507__C11-C14__ratio                 29
507__C1-C14__ratio                  24
507__C1-C14__ratio__ProductCD-C     22
507__C11-C14__ratio__ProductCD-C    10
502__C1__ProductCD-C                 8
507__C1-C14__diff                    4
raw__V187                            4
raw__C1                              2
507__C12-C14__ratio                  2
508__C6-D10__ratio                   1
507__C10-C14__ratio                  1
509__C1-V78__ratio__ProductCD-C      1
507__C1-C14__diff__ProductCD-C       1
raw__D2                              1
raw__V45                             1
507__C1-C4__ratio__ProductCD-C       1
raw__V242                            1
507__C1-C12__ratio                   1
502__cnt__C1                         1
dtype: int64

In [109]:
import shutil

list_from_dir = ['raw_use' , 'org_use']
list_from_dir += ['raw_trush' , 'org_trush', 'valid_trush']
to_dir = 'valid'
list_not_found = []

for feature in list_good_feature:
    for from_dir in list_from_dir:
        
        path_train_feature = f"../feature/{from_dir}/{feature}_train.gz"
        path_test_feature = f"../feature/{from_dir}/{feature}_test.gz"
        
        if os.path.exists(path_train_feature) and os.path.exists(path_test_feature):
#         if os.path.exists(path_train_feature):
#         if os.path.exists(path_test_feature):
            try:
                shutil.move(path_train_feature, f'../feature/{to_dir}')
                shutil.move(path_test_feature, f'../feature/{to_dir}')
            except FileNotFoundError:
                print(feature)
                list_not_found.append(feature)